## CAO Points Analysis

***

In [58]:
# To access a URL from python
# import urllib

# Convenient HTTP requests
import requests as rq

# Regular expressions
import re

# Should always call this out on the requirements txt file as it doesn't come as standard package with python
# Only if you have anaconda does it come with the package

# Dates and times
import datetime as dt

# Data frames
import pandas as pd

# for downloading 
import urllib.request as urlrq

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Get the current date and time
now = dt.datetime.now()

# Format as a string
nowstr = now.strftime('%Y%m%d_%H%M%S')

<br>

## 2021 Points 
https://www.cao.ie/index.php?page=points&p=2021&bb=points
***

In [3]:
# Fetch the CAO points URL.
resp = rq.get('http://www2.cao.ie/points/l8.php')
# Have a quick peek (http code 200 means it is ok)
resp
# To see the whole text
# resp.text

<Response [200]>

#### Save Original data set

***

In [4]:
# Create a file path for the original data
pathhtml = 'data/cao2021_' + nowstr + '.html'

<br>

**Error on server**

Technically, the server says we should decode as per:
```
Content-Type: text/html; charset=iso-8859-1
```
However, one line uses \x96 which isn't defined in iso-8859-1.
<br>Therefore we use the similar decoding standard cp1252, which is very similar but includes #x96.

In [5]:
# The server uses the wrong encoding, fix it
original_encoding = resp.encoding
# Change to cp1252
resp.encoding = 'cp1252'

In [6]:
# Save the original html file
with open (pathhtml, 'w') as f:
    f.write(resp.text)

<br>

### Use regular expressions to select lines we want

***

In [7]:
# Compile the regular expression for matching lines
# re_course = re.compile('([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *')
re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

<br>

### Loop through the lines of the response

***

In [8]:
def points_to_array(s):
    portfolio = ''
    if s[0] == '#':
        portfolio = '#'
    random = ''
    if s[-1] == '*':
        random = '*'
    points = ''
    for i in s:
        if i.isdigit():
            points = points + i
    return [points, portfolio, random]

In [9]:
# The file path for the csv file
path2021 = 'data/cao2021_csv_' + nowstr + '.csv'

In [10]:
# Keep track of how many courses we process 
no_lines = 0

# Open the cav file for writing
with open (path2021, 'w') as f:
    # Write a header row.
    f.write(','.join(['code','title','pointsR1','pointsR2']) + '\n')
    # Loop through lines of the respone
    for line in resp.iter_lines():
        # Decode the line, changed from ISO-8859-1 to cp1252
        # as one line uses \x96 which isn't defined in iso-8859-1
        dline = line.decode('cp1252')
        # Match only the lines representing courses
        if re_course.fullmatch(dline):
            # Add one to the lines counter
            no_lines = no_lines + 1
            # Uncomment next lines to see the original
            # print(line)
            # Pick out the relevant parts of the matched line
            # csv_version = re_course.sub(r'\1,\2,\3,\4', dline)
            # Print the CSV-style line
            # print(csv_version)
            # Split the line on two or more spaces
            # linesplit = re.split('  +', dline)
            # print(linesplit)
            # print(','.join(linesplit))
            # Rejoin the substrings with commas between them
            # Debug
            # print(len(linesplit), linesplit, dline)
            # The course code
            course_code = dline[:5]
            # print(course_code)
            # The course title
            course_title = dline[7:57].strip()
            # print(f"'{course_title}'")
            # Round one points
            course_points = re.split(' +', dline[60:])
            if len(course_points) !=2:
                # print(f"'{course_code} {course_points}'")
                course_points = course_points[:2]
            # Join the fields using a comma
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between
            f.write(','.join(linesplit) + '\n')

# Print the total number of processed lines
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


<br>

**NB:** It was verified as of 24th Nov 2021 that there was 949 courses exactly in the CAO 2021 point list

In [11]:
df2021 = pd.read_csv(path2021, encoding='cp1252')

In [12]:
df2021

,code,title,pointsR1,pointsR2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


<br>

## 2020 Points
https://www.cao.ie/index.php?page=points&p=2020&bb=points

***

In [13]:
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

<br>

#### Save Original File

***

In [14]:
# Create a file path for the original data
pathxlsx = 'data/cao2020_' + nowstr + '.xlsx'

In [15]:
# Save original file to disk
urlrq.urlretrieve(url2020, pathxlsx)

('data/cao2020_20211207_194337.xlsx',
 <http.client.HTTPMessage at 0x1e6ff9611c0>)

<br>

#### Load Spreadsheet using pandas

***

In [16]:
# Download and parse the excel spreadsheet
df2020 = pd.read_excel(url2020, skiprows=10)

In [17]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Spot check random row
df2020.iloc[1463]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [19]:
# Spot check the 4th last row
df2020.iloc[-4]

CATEGORY (i.e.ISCED description)    Information and Communication Technologies (ICTs)
COURSE TITLE                                             Software Systems Development
COURSE CODE2                                                                    WD210
R1 POINTS                                                                         279
R1 Random *                                                                       NaN
R2 POINTS                                                                         NaN
R2 Random*                                                                        NaN
EOS                                                                               279
EOS Random *                                                                      NaN
EOS Mid-point                                                                     337
LEVEL                                                                               8
HEI                                                 Wa

In [20]:
# Create a file path for the pandas data
path2020 = 'data/cao2020_' + nowstr + '.csv'

In [21]:
# Save pandas data frame to disk
df2020.to_csv(path2020) 

In [22]:
######################## testing

# Check the initial shape of the DataFrame
df2020.shape

(1464, 23)

In [148]:
######################## testing

## https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/
    
# Delete rows where case numbers are zero
# This deletion is completed by "selecting" rows where case numbers are non zero
df2020new = df2020.loc[df2020["LEVEL"] > 7]
df2020new.shape

(1027, 23)

<br>

## 2019 Points 
https://www.cao.ie/index.php?page=points&p=2019&bb=points
***

##### Steps to reproduce

1. Download original pdf file.
2. Open original pdf file in Microsoft Word.
3. Save Microsoft Word's converted PDF in docx format.
4. Re-save Word document for editing.
5. Delete headers and footers.
6. Delete preamble on page 1.
7. Select all and copy.
8. Paste into Notepad ++.
9. Remove HEI name headings and paste onto each course line. 
10. Delete blank lines.
11. Replace double tab characters with a single. 
12. Deleted tab character at end of line 308. 
13. Change backticks to apostrophes.

In [24]:
df2019 = pd.read_csv('data/cao2019_20211109_edited.csv', header=0, sep='\t')

In [25]:
# delimiter=';'
# df2019 = pd.read_csv('data/cao2019_20211109_edited.csv', sep='\t')

In [26]:
df2019

,HEI,Course Code,INSTITUTION and COURSE,EOS,Mid
0,Athlone Institute of Technology,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,Athlone Institute of Technology,AL802,Software Design with Cloud Computing,301,306.0
2,Athlone Institute of Technology,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,Athlone Institute of Technology,AL805,Network Management and Cloud Infrastructure,329,442.0
4,Athlone Institute of Technology,AL810,Quantity Surveying,307,349.0
...,...,...,...,...,...
925,Waterford Institute of Technology,WD200,Arts (options),221,296.0
926,Waterford Institute of Technology,WD210,Software Systems Development,271,329.0
927,Waterford Institute of Technology,WD211,Creative Computing,275,322.0
928,Waterford Institute of Technology,WD212,Recreation and Sport Management,274,311.0


In [27]:
# Create a file path for the pandas data
path2019 = 'data/cao2019_' + nowstr + '.csv'

In [28]:
# Save pandas data frame to disk
df2019.to_csv(path2019) 

<br>

##  Concat and join

In [29]:
courses2021 = df2021[['code', 'title']]
courses2021

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
944,WD211,Creative Computing
945,WD212,Recreation and Sport Management
946,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [30]:
courses2020 = df2020new[['COURSE CODE2', 'COURSE TITLE']]
courses2020.columns = ['code', 'title']
courses2020

,code,title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
1455,WD200,Arts (options)
1460,WD210,Software Systems Development
1461,WD211,Creative Computing
1462,WD212,Recreation and Sport Management


In [31]:
courses2019 = df2019[['Course Code', 'INSTITUTION and COURSE']]
courses2019.columns = ['code', 'title']
courses2019

,code,title
0,AL801,Software Design with Virtual Reality and Gaming
1,AL802,Software Design with Cloud Computing
2,AL803,Software Design with Mobile Apps and Connected...
3,AL805,Network Management and Cloud Infrastructure
4,AL810,Quantity Surveying
...,...,...
925,WD200,Arts (options)
926,WD210,Software Systems Development
927,WD211,Creative Computing
928,WD212,Recreation and Sport Management


In [32]:
allcourses = pd.concat([courses2021, courses2020, courses2019], ignore_index=True)
allcourses

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
2901,WD200,Arts (options)
2902,WD210,Software Systems Development
2903,WD211,Creative Computing
2904,WD212,Recreation and Sport Management


In [33]:
allcourses.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2901     True
2902     True
2903     True
2904     True
2905     True
Length: 2906, dtype: bool

In [34]:
allcourses.sort_values('code')

,code,title
175,AC120,International Business
2144,AC120,International Business
949,AC120,International Business
176,AC137,Liberal Arts
2145,AC137,Liberal Arts
...,...,...
1975,WD230,Mechanical and Manufacturing Engineering
946,WD230,Mechanical and Manufacturing Engineering
2905,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [35]:
################ testing

dfallcourses = allcourses

In [36]:
################## testing

# Create a file path for the pandas data
pathallcourses = 'data/allcourses_' + nowstr + '.csv'

In [37]:
################## testing

# Save pandas data frame to disk
dfallcourses.to_csv(pathallcourses) 

In [38]:
allcourses.loc[175], allcourses.loc[949]

(code                      AC120
 title    International Business
 Name: 175, dtype: object,
 code                      AC120
 title    International Business
 Name: 949, dtype: object)

In [39]:
# Finds all extra copies of duplicated rows
allcourses[allcourses.duplicated()]

,code,title
949,AC120,International Business
950,AC137,Liberal Arts
952,AD102,Graphic Design and Moving Image Design (portfo...
955,AD204,Fine Art (portfolio)
956,AD211,Fashion Design (portfolio)
...,...,...
2901,WD200,Arts (options)
2902,WD210,Software Systems Development
2903,WD211,Creative Computing
2904,WD212,Recreation and Sport Management


In [40]:
# Returns a copy of the data frame with the duplicates removed
allcourses.drop_duplicates()

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
2844,TL801,Animation Visual Effects and Motion Design
2845,TL802,"TV, Radio and New Media"
2846,TL803,Music Technology
2849,TL812,Computing with Digital Media


In [41]:
# duplicates on the course code
allcourses[allcourses.duplicated(subset=['code'])]

,code,title
949,AC120,International Business
950,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
952,AD102,Graphic Design and Moving Image Design (portfo...
953,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
2901,WD200,Arts (options)
2902,WD210,Software Systems Development
2903,WD211,Creative Computing
2904,WD212,Recreation and Sport Management


In [42]:
# Returns a copy of the data frame with the duplicates removed - based only on code
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)
allcourses

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
1209,SG441,Environmental Science
1210,SG446,Applied Archaeology
1211,TL803,Music Technology
1212,TL812,Computing with Digital Media


<br>

### Join to the points

***

In [43]:
# Set the index to the code column
df2021.set_index('code', inplace=True)
df2021.columns = ['title', 'points_r1_2021', 'points_r2_2021']
df2021

,title,points_r1_2021,points_r2_2021
code,,,
AL801,Software Design for Virtual Reality and Gaming,300,NaN
AL802,Software Design in Artificial Intelligence for...,313,NaN
AL803,Software Design for Mobile Apps and Connected ...,350,NaN
AL805,Computer Engineering for Network Infrastructure,321,NaN
AL810,Quantity Surveying,328,NaN
...,...,...,...
WD211,Creative Computing,270,NaN
WD212,Recreation and Sport Management,262,NaN
WD230,Mechanical and Manufacturing Engineering,230,230


In [44]:
# Set the index to the code column
allcourses.set_index('code', inplace=True)
allcourses

,title
code,
AL801,Software Design for Virtual Reality and Gaming
AL802,Software Design in Artificial Intelligence for...
AL803,Software Design for Mobile Apps and Connected ...
AL805,Computer Engineering for Network Infrastructure
AL810,Quantity Surveying
...,...
SG441,Environmental Science
SG446,Applied Archaeology
TL803,Music Technology


In [45]:
allcourses = allcourses.join(df2021[['points_r1_2021']])
allcourses

,title,points_r1_2021
code,,
AL801,Software Design for Virtual Reality and Gaming,300
AL802,Software Design in Artificial Intelligence for...,313
AL803,Software Design for Mobile Apps and Connected ...,350
AL805,Computer Engineering for Network Infrastructure,321
AL810,Quantity Surveying,328
...,...,...
SG441,Environmental Science,NaN
SG446,Applied Archaeology,NaN
TL803,Music Technology,NaN


In [46]:
df2020_r1 = df2020new[['COURSE CODE2', 'R1 POINTS']]
df2020_r1.columns = ['code', 'points_r1_2020']
df2020_r1

,code,points_r1_2020
0,AC120,209
1,AC137,252
2,AD101,#+matric
3,AD102,#+matric
4,AD103,#+matric
...,...,...
1455,WD200,AQA
1460,WD210,279
1461,WD211,271
1462,WD212,270


In [47]:
# Set the index to the code column
df2020_r1.set_index('code', inplace=True)
df2020_r1

,points_r1_2020
code,
AC120,209
AC137,252
AD101,#+matric
AD102,#+matric
AD103,#+matric
...,...
WD200,AQA
WD210,279
WD211,271


In [48]:
# Join 2020 points to allcourses
allcourses = allcourses.join(df2020_r1)
allcourses

,title,points_r1_2021,points_r1_2020
code,,,
AL801,Software Design for Virtual Reality and Gaming,300,303
AL802,Software Design in Artificial Intelligence for...,313,332
AL803,Software Design for Mobile Apps and Connected ...,350,337
AL805,Computer Engineering for Network Infrastructure,321,333
AL810,Quantity Surveying,328,319
...,...,...,...
SG441,Environmental Science,NaN,NaN
SG446,Applied Archaeology,NaN,NaN
TL803,Music Technology,NaN,NaN


In [49]:
df2019_r1 = df2019[['Course Code', 'EOS']]
df2019_r1.columns = ['code', 'points_r1_2019']
df2019_r1

,code,points_r1_2019
0,AL801,304
1,AL802,301
2,AL803,309
3,AL805,329
4,AL810,307
...,...,...
925,WD200,221
926,WD210,271
927,WD211,275
928,WD212,274


In [50]:
# Set the index to the code column
df2019_r1.set_index('code', inplace=True)
df2019_r1

,points_r1_2019
code,
AL801,304
AL802,301
AL803,309
AL805,329
AL810,307
...,...
WD200,221
WD210,271
WD211,275


In [51]:
# Join 2019 points to allcourses
allcourses = allcourses.join(df2019_r1)
allcourses

,title,points_r1_2021,points_r1_2020,points_r1_2019
code,,,,
AL801,Software Design for Virtual Reality and Gaming,300,303,304
AL802,Software Design in Artificial Intelligence for...,313,332,301
AL803,Software Design for Mobile Apps and Connected ...,350,337,309
AL805,Computer Engineering for Network Infrastructure,321,333,329
AL810,Quantity Surveying,328,319,307
...,...,...,...,...
SG441,Environmental Science,NaN,NaN,297
SG446,Applied Archaeology,NaN,NaN,289
TL803,Music Technology,NaN,NaN,264


In [52]:
################ testing

dfround1 = allcourses

In [53]:
################## testing

# Create a file path for the pandas data
pathround1 = 'data/round1_' + nowstr + '.csv'

In [54]:
################## testing

# Save pandas data frame to disk
dfround1.to_csv(pathround1) 

In [138]:
# https://www.geeksforgeeks.org/python-pandas-dataframe-replace/
dfround1a = dfround1.replace('[*,#]', '', regex = True)

In [139]:
pathround1 = 'data/round1a_' + nowstr + '.csv'
dfround1a.to_csv(pathround1) 

In [140]:
# https://www.statology.org/create-column-based-on-condition-pandas/

#create new column titled 'assist_more'
dfround1a['Change 21 vs 20'] = np.where(dfround1a['points_r1_2021']>dfround1a['points_r1_2021'], 'Points increase', 'Points decrease')

#view DataFrame 
dfround1a

,title,points_r1_2021,points_r1_2020,points_r1_2019,Change 21 vs 20
code,,,,,
AL801,Software Design for Virtual Reality and Gaming,300,303,304,Points decrease
AL802,Software Design in Artificial Intelligence for...,313,332,301,Points decrease
AL803,Software Design for Mobile Apps and Connected ...,350,337,309,Points decrease
AL805,Computer Engineering for Network Infrastructure,321,333,329,Points decrease
AL810,Quantity Surveying,328,319,307,Points decrease
...,...,...,...,...,...
SG441,Environmental Science,NaN,NaN,297,Points decrease
SG446,Applied Archaeology,NaN,NaN,289,Points decrease
TL803,Music Technology,NaN,NaN,264,Points decrease


In [143]:
dfround1b = dfround1a.astype({"points_r1_2021": Int64, "points_r1_2021": Int64})

NameError: name 'Int64' is not defined

In [145]:
dfround1a['points_r1_2021'] = dfround1a['points_r1_2021'].astype('Int64')

TypeError: object cannot be converted to an IntegerDtype

In [146]:
dfround1a['points_r1_2021'] = dfround1a['points_r1_2021'].fillna(0).astype('int')

ValueError: invalid literal for int() with base 10: ''

In [124]:
dfround1b.dtypes

title              object
points_r1_2021     object
points_r1_2020     object
points_r1_2019     object
Change             object
Change 21 vs 20    object
dtype: object

In [112]:
dfround1a.apply(pd.to_numeric, errors='ignore')

,title,points_r1_2021,points_r1_2020,points_r1_2019,Change,Change 21 vs 20
code,,,,,,
AL801,Software Design for Virtual Reality and Gaming,300,303,304,Points decrease,Points decrease
AL802,Software Design in Artificial Intelligence for...,313,332,301,Points decrease,Points decrease
AL803,Software Design for Mobile Apps and Connected ...,350,337,309,Points decrease,Points decrease
AL805,Computer Engineering for Network Infrastructure,321,333,329,Points decrease,Points decrease
AL810,Quantity Surveying,328,319,307,Points decrease,Points decrease
...,...,...,...,...,...,...
SG441,Environmental Science,NaN,NaN,297,Points decrease,Points decrease
SG446,Applied Archaeology,NaN,NaN,289,Points decrease,Points decrease
TL803,Music Technology,NaN,NaN,264,Points decrease,Points decrease


In [113]:
dfround1a.dtypes

title              object
points_r1_2021     object
points_r1_2020     object
points_r1_2019     object
Change             object
Change 21 vs 20    object
dtype: object

In [114]:
dfround1a[["points_r1_2021", "points_r1_2020"]] = dfround1a[["points_r1_2021", "points_r1_2020"]].apply(pd.to_numeric, errors='ignore')

In [115]:
dfround1a.dtypes

title              object
points_r1_2021     object
points_r1_2020     object
points_r1_2019     object
Change             object
Change 21 vs 20    object
dtype: object

In [131]:
dfround1a['change1'] = (dfround1a[{str(int(startamt)}]) - (dfround1a['points_r1_2021'])

SyntaxError: invalid syntax (<ipython-input-131-9a6cd722a293>, line 1)

In [134]:
dfround1a['change1'] = (dfround1a['points_r1_2021'].astype(int)) - (dfround1a['points_r1_2021'].astype(int))

ValueError: cannot convert float NaN to integer

In [105]:
import pandas as pd

items_df = pd.DataFrame({
    'Id': [302, 504, 708, 103, 343, 565],
    'Name': ['Watch', 'Camera', 'Phone', 'Shoes', 'Laptop', 'Bed'],
    'Actual Price': [300, 400, 350, 100, 1000, 400],
    'Discount(%)': [10, 15, 5, 0, 2, 7]
})

print("Initial DataFrame:")
print(items_df, "\n")

items_df['Final Price'] = items_df['Actual Price'] - ((items_df['Discount(%)']/100) * items_df['Actual Price'])


print("DataFrame after addition of new column")
print(items_df, "\n")

items_df.dtypes

Initial DataFrame:
    Id    Name  Actual Price  Discount(%)
0  302   Watch           300           10
1  504  Camera           400           15
2  708   Phone           350            5
3  103   Shoes           100            0
4  343  Laptop          1000            2
5  565     Bed           400            7 

DataFrame after addition of new column
    Id    Name  Actual Price  Discount(%)  Final Price
0  302   Watch           300           10        270.0
1  504  Camera           400           15        340.0
2  708   Phone           350            5        332.5
3  103   Shoes           100            0        100.0
4  343  Laptop          1000            2        980.0
5  565     Bed           400            7        372.0 



Id                int64
Name             object
Actual Price      int64
Discount(%)       int64
Final Price     float64
dtype: object

In [78]:
# create a list of our conditions
conditions = [
    (dfround1a['points_r1_2021-points_r1_2020'] > 0),
    (dfround1a['points_r1_2021-points_r1_2020'] < 0),
    ]

# create a list of the values we want to assign for each condition
values = ['Points increased', 'Points unchanged', 'Points decreased']

# create a new column and use np.select to assign values to it using our lists as arguments
df['points'] = np.select(conditions, values)

# display updated DataFrame
df.head()

KeyError: 'points_r1_2021-points_r1_2020'

In [126]:
startamt = "52"
userturn = "9"
print(f"there are {str(int(startamt) - int(userturn))} balls left")

there are 43 balls left


***
### End